In [1]:
import pandas as pd
import json
import folium
from csv import reader
import psutil

In [2]:
df = pd.DataFrame()

chunk_reader = pd.read_csv(
    "/var/local/mobile_scan_results.csv", 
    delimiter=";", 
    chunksize=100000, 
    usecols=["created_at", "latitude", "longitude", "raw_decoded_message"]
)

for i, chunk_df in enumerate(chunk_reader):
    chunk_df = chunk_df.dropna(
        subset=["latitude", "longitude", "raw_decoded_message"]
    ).assign(
        scanned_ap=lambda df: (df.raw_decoded_message.apply(lambda row: (json.loads(row)["scan_result"]["scanned_aps"])))
    ).drop(
        "raw_decoded_message", axis=1
    ).explode(
        "scanned_ap"
    )

    # normalize the json column into multiple columns, merge it back to the original df, and add some sanitization
    chunk_df = chunk_df.merge(
        pd.json_normalize(chunk_df["scanned_ap"], ).loc[:, ["ssid", "bssid", "level"]], 
        left_index=True, 
        right_index=True
    ).drop(
        "scanned_ap", axis=1
    ).assign(
        latitude=lambda df: df.latitude.round(5), 
        longitude=lambda df: df.longitude.round(5)
    ).drop_duplicates(subset=["bssid", "level", "latitude", "longitude"], keep="first", ignore_index=True)

    df = pd.concat([df, chunk_df], ignore_index=True).drop_duplicates(subset=["bssid", "level", "latitude", "longitude"], keep="first", ignore_index=True)

    
    if (psutil.virtual_memory().available * 100 / psutil.virtual_memory().total) >= 90.0:
        print("Memory Limit!!")
        break

    print(df.index.size)

df

36351
68521
85607
111572
135336
163654
189067
203358
221789
245029
288460
336970
385843
450651
500171


,created_at,latitude,longitude,ssid,bssid,level,frequency
0,2024-03-31 17:17:31.209511,47.58812,-122.30138,TRENDnet740_39HD,3c:8c:f8:ed:b5:cf,-72,2412
1,2024-03-31 17:17:34.254326,47.58812,-122.30138,SGS Staff,78:45:58:ec:08:1b,-78,2462
2,2024-03-31 17:17:28.189099,47.58812,-122.30138,SGS Staff,78:45:58:ec:00:9b,-80,2437
3,2024-03-31 17:21:13.905929,47.58730,-122.30161,CenturyLink3940,5c:64:8e:bb:5c:55,-80,2412
4,2024-03-31 17:21:10.847109,47.58730,-122.30161,FieldHouse,d4:6e:0e:35:c1:12,-80,2462
...,...,...,...,...,...,...,...
500166,2024-12-11 01:55:15.196344,43.46807,-84.02868,ANTHC,10:0c:6b:80:aa:cb,-64,2447
500167,2024-12-11 01:55:12.13658,43.46807,-84.02868,InternetOnly,b8:38:61:e3:cf:54,-81,2437
500168,2024-12-11 01:55:21.276356,43.46878,-84.02865,InternetOnly,74:11:b2:6d:fe:44,-75,5220
500169,2024-12-11 01:55:30.609371,43.46885,-84.02785,ANTHC-SECURE,74:11:b2:6d:fe:42,-75,5220


In [3]:
df.to_csv("scanned_aps_sanitized.csv")